## Print POD

In [1]:
import os
import re
import win32api
import win32print
from datetime import datetime

# Printing Function
def print_pdf(path):
    try:
        win32api.ShellExecute(0, "print", path, None, ".", 0)
        print(f"✅ Sent to printer: {os.path.basename(path)}")
    except Exception as e:
        print(f"❌ Failed to print {path}: {e}")

# ───────────────────────────────
# 1.  Folder with POD PDFs
# ───────────────────────────────
FOLDER = r"\\Quickbook2024\d\Drive D\QuickBooks\3- Year 2025\Purchase Order (Vendor)- POD"

# ───────────────────────────────
# 2.  Copy-&-paste POD list here
# ───────────────────────────────
raw_input = """
POD-250957, POD-251033, POD-251058, POD-251054, POD-251061, POD-251129, POD-251140, POD-251162, POD-251178, POD-251206, POD-251215, POD-251223, POD-251227, POD-251243, POD-251244, POD-251245, POD-251276, POD-251277, POD-251272, POD-251273, POD-251288, POD-251294, POD-251297, POD-251298, POD-251299, POD-251301, POD-251331    
"""

# ───────────────────────────────
# 3.  Parse & numerically sort PODs
# ───────────────────────────────
pod_list = [p.strip() for p in raw_input.replace("\n", "").split(",") if p.strip()]
PODS_ORDERED = sorted(pod_list, key=lambda x: int(x.split("-")[1]))  # ascending 250382 → 250443 …
PODS_SET     = set(PODS_ORDERED)   # fast membership test


# Track latest file per POD
latest_signed_files = {}

for pod in PODS_SET:
    latest_time = None
    latest_file = None

    for filename in os.listdir(FOLDER):
        if pod in filename:
            full_path = os.path.join(FOLDER, filename)
            modified_time = os.path.getmtime(full_path)

            if latest_time is None or modified_time > latest_time:
                latest_time = modified_time
                latest_file = full_path

    if latest_file:
            latest_signed_files[pod] = latest_file
            print(f"✅ Latest for {pod}: {latest_file} (Modified: {datetime.fromtimestamp(latest_time)})")

            print_pdf(latest_file) # Printing
    else:
        print(f"❌ No signed version found for {pod}")

✅ Latest for POD-251227: \\Quickbook2024\d\Drive D\QuickBooks\3- Year 2025\Purchase Order (Vendor)- POD\POD-251227_NTA_Purdue_SO-20251215.pdf (Modified: 2025-08-28 12:18:06.986248)
✅ Sent to printer: POD-251227_NTA_Purdue_SO-20251215.pdf
✅ Latest for POD-251331: \\Quickbook2024\d\Drive D\QuickBooks\3- Year 2025\Purchase Order (Vendor)- POD\POD-251331_NTA_CoastIPC_Urgent.pdf (Modified: 2025-09-16 23:19:03.021786)
✅ Sent to printer: POD-251331_NTA_CoastIPC_Urgent.pdf
✅ Latest for POD-251206: \\Quickbook2024\d\Drive D\QuickBooks\3- Year 2025\Purchase Order (Vendor)- POD\POD-251206_NTA_Neya_SO_20251198.pdf (Modified: 2025-08-24 22:26:51.535484)
✅ Sent to printer: POD-251206_NTA_Neya_SO_20251198.pdf
✅ Latest for POD-251272: \\Quickbook2024\d\Drive D\QuickBooks\3- Year 2025\Purchase Order (Vendor)- POD\POD-251272_NTA_Ridgeline Engineering_SO-20251251.pdf (Modified: 2025-09-08 02:35:52.053943)
✅ Sent to printer: POD-251272_NTA_Ridgeline Engineering_SO-20251251.pdf
✅ Latest for POD-251243: \\Q

# Tariff Calculator

In [3]:
import pdfplumber
import pandas as pd
import re
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill
from tabulate import tabulate

# === TARIFF Model ===
def calculate_tariff(hs_code: str, origin: str) -> float:
    hs_code = hs_code.replace('.', '').strip()
    origin = origin.strip().upper()

    if hs_code.startswith('847141'):
        return 0.0
    elif hs_code.startswith('847330'):
        return 45.0 if origin == 'CHINA' else 0.0
    elif hs_code.startswith('850440'):
        return 10.0 if origin == 'TAIWAN' else 55.0 if origin == 'CHINA' else 0.0
    elif hs_code.startswith('854442'):
        return 22.6 if origin == 'TAIWAN' else 57.6 if origin == 'CHINA' else 0.0
    elif hs_code.startswith('852910'):
        return 20.0 if origin == 'TAIWAN' else 30.0 if origin == 'CHINA' else 0.0
    elif hs_code.startswith('852589'):
        return 20.0 if origin == 'TAIWAN' else 55.0 if origin == 'CHINA' else 0.0
    return 0.0

# === BLOCK PARSER ===
def parse_item_block(block_lines):
    first_line = block_lines[0]
    item_name = None
    amount = None
    country = "NA"
    hs_code = "NA"
    htsus = "NA"

    match = re.search(r'POD-\d{6,7}[A-Z()]*\s+(.*?)\s+\d+\s+PCS\s+([\d,]+)', first_line)
    if match:
        raw_item = match.group(1).strip()
        item_name = re.sub(r'\s+\d+$', '', raw_item)
        amount = match.group(2).replace(",", "")
    else:
        return None

    for line in block_lines:
        if "Country of Origin" in line:
            origin_match = re.search(r"Country of Origin\s*[:：]?\s*([A-Z]+)", line, re.IGNORECASE)
            if origin_match:
                country = origin_match.group(1).upper()
        if "HS Code" in line:
            hs_match = re.search(r"HS Code\s*[:：]?\s*(\d+)", line)
            if hs_match:
                hs_code = hs_match.group(1)
        if "HTSUS" in line:
            htsus_match = re.search(r"HTSUS\s*[:：]?\s*([\d\.]+)", line)
            if htsus_match:
                htsus = htsus_match.group(1)

    tariff = calculate_tariff(hs_code, country)

    return {
        "Item": item_name,
        "Amount": amount,
        "Country of Origin": country,
        "HS Code": hs_code,
        "HTSUS": htsus,
        "Tariff (%)": tariff
    }

# === PDF PARSER ===
def extract_items_from_invoice(pdf_path):
    items = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            lines = page.extract_text().split('\n') if page.extract_text() else []
            current_block = []
            for line in lines:
                if 'PCS' in line:
                    if current_block:
                        item = parse_item_block(current_block)
                        if item:
                            items.append(item)
                    current_block = [line]
                else:
                    if current_block:
                        current_block.append(line)
            if current_block:
                item = parse_item_block(current_block)
                if item:
                    items.append(item)

    df = pd.DataFrame(items)
    if not df.empty:
        df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce').fillna(0)
        df['Tariff Amount'] = df['Amount'] * df['Tariff (%)'] / 100
        total_tariff = df['Tariff Amount'].sum()
        service_fee = df['Amount'].sum() * 0.003464
        service_fee = min( service_fee, 634.62)   # max service fee is 634.62
        total_fee = total_tariff + service_fee
        return df, total_tariff, service_fee, total_fee
    return df, 0.0, 0.0, 0.0

# === EXCEL WRITER ===
def write_tariff_to_excel(df, total_tariff, service_fee, total_fee, output_path):
    df.to_excel(output_path, index=False)

    wb = load_workbook(output_path)
    ws = wb.active

    start_row = ws.max_row + 2
    labels = ["TOTAL TARIFF", "MERCHANDISE SERVICE FEE", "TOTAL FEE"]
    values = [total_tariff, service_fee, total_fee]
    fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

    for i, (label, value) in enumerate(zip(labels, values)):
        row = start_row + i
        ws.cell(row=row, column=1, value=label).font = Font(bold=True)
        ws.cell(row=row, column=1).fill = fill
        cell = ws.cell(row=row, column=2, value=value)
        cell.font = Font(bold=True)
        cell.number_format = '"$"#,##0.00'
        cell.fill = fill

    wb.save(output_path)
    print(f"✅ Full tariff table with summary saved to: {output_path}")

# === USAGE ===
pdf_path = r"C:\Users\Admin\OneDrive - neousys-tech\Desktop\Invoice\IN.PL_IN250924008_NTA.pdf"
output_path = r"C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\06 Payment\APCC-W250800_Entry Summary_IN250_traiff excel.xlsx"

df_items, total_tariff, service_fee, total_fee = extract_items_from_invoice(pdf_path)
write_tariff_to_excel(df_items, total_tariff, service_fee, total_fee, output_path)
print(f'Total Items: {len(df_items)}')

print("\n" + "=" * 60)
print("\033[1;44m\033[1;37m     🚢  TARIFF CALCULATION SUMMARY  📦     \033[0m")
print("=" * 60)

print(f"\033[1;33m🔶 Total Tariff :\033[0m   \033[1m${total_tariff:,.2f}\033[0m")
print(f"\033[1;36m🔷 Service Fee  :\033[0m   \033[1m${service_fee:,.2f}\033[0m")
print(f"\033[1;32m🟩 Entry Fee    :\033[0m   \033[1m${total_fee:,.2f}\033[0m")

print("=" * 60 + "\n")

print("\033[1;34m📄 Tariff Breakdown Table:\033[0m")
print(tabulate(df_items, headers='keys', tablefmt='fancy_grid', showindex=True))


✅ Full tariff table with summary saved to: C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\06 Payment\APCC-W250800_Entry Summary_IN250_traiff excel.xlsx
Total Items: 51

     🚢  TARIFF CALCULATION SUMMARY  📦     
🔶 Total Tariff :   $8,653.02
🔷 Service Fee  :   $634.62
🟩 Entry Fee    :   $9,287.64

📄 Tariff Breakdown Table:
╒════╤═════════════════════════════════════════╤══════════╤═════════════════════╤═══════════╤════════════╤══════════════╤═════════════════╕
│    │ Item                                    │   Amount │ Country of Origin   │   HS Code │ HTSUS      │   Tariff (%) │   Tariff Amount │
╞════╪═════════════════════════════════════════╪══════════╪═════════════════════╪═══════════╪════════════╪══════════════╪═════════════════╡
│  0 │ NRU-230V-AWP-JAO64G-AI01-25             │    95125 │ TAIWAN              │    847150 │ 9903.01.32 │          0   │           0     │
├────┼─────────────────────────────────────────┼──────────┼─────────────────────┼───────────┼───────────

# Packinglist Extraction

In [16]:
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        print(page.extract_text())

Invoice
Neousys Technology Inc.
11F., No.198, Jian 8th Rd.,
Zhonghe District, New Taipei City 235042
Taiwan
Phone:+886-2-22236182 FAX:+886-2-22236183
INV No.: IN250917003
INV Date: Sep 17, 2025.
Bill to : Ship to :
Neousys Technology America, Inc. Neousys Technology America, Inc.
55 East Hintz Road, Wheeling, IL 60090 55 East Hintz Road, Wheeling, IL 60090
United States United States
ATTN : Roy Wang ATTN : Roy Wang
Shipping Method: SPEEDMARK
Customer PO No. : POD-250847, POD-250955, POD-251057, POD-251094, POD-251109, POD- Currency : USD
251215, POD-251217, POD-251231, POD-251248, POD-251250, POD-251235, POD-251256, POD-
251249
Description of Industrial Computer or Computer
Item Cust. PO No. Unit Price QTY Unit Amount
accessory
1 POD-250847 N8108GC-QD-i7G965-A4-MII-CF4-50 2755 20 PCS 55100
Nuvo-8108GC-QD-A6, including PCIe-PoE354at-1, i7-
9700E, 2x DDR4-16GB-32-SM, SSD-1TB-TLC5ET-
TD, GC-RTXA4000-LT1
Country of Origin :TAIWAN
HS Code :847150
HTSUS: 9903.01.32
2 POD-250955 Cblkit-FP-NRU

In [17]:
import pdfplumber
import re
import pandas as pd

# === Pattern checkers ===
def is_box_header(line):
    return re.match(r"^\d+-\s*\d+", line)

def is_item_line(line):
    line = line.strip()
    if "TOTAL" in line.upper():
        return False
    return bool(re.search(r"\d+\s+\d+\.\d{2}\b", line))

def extract_box_number(line):
    match = re.match(r"^\d+-\s*\d+(?:-\d+)?", line)
    return match.group(0) if match else None

def extract_item_qty(line):
    # Remove Box No. prefix if present (e.g., "47- 2" or "47- 3-9")
    line_clean = re.sub(r'^\d+-\s*\d+(?:-\d+)?\s*', '', line)
    match = re.search(r'(?P<item>.+?)\s+(?P<qty>\d+)\s+\d+\.\d{2}', line_clean)
    if match:
        return match.group('item').strip(), int(match.group('qty'))
    return None, None

# === Parse PDF ===
invoice_lines = []
packing_lines = []
in_packing_list = False  # Switch flips at first box header

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        for line in page.extract_text().split('\n'):
            if not in_packing_list and is_box_header(line):
                in_packing_list = True
                packing_lines.append(line)
            elif in_packing_list and (is_box_header(line) or is_item_line(line)):
                packing_lines.append(line)
            elif not in_packing_list and is_item_line(line):
                invoice_lines.append(line)

# === Build DataFrames ===
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

df_invoice = pd.DataFrame({"Invoice Lines": invoice_lines})

df_packing = pd.DataFrame({"Packing List Lines": packing_lines})
df_packing["Box No."] = df_packing["Packing List Lines"].apply(extract_box_number)
df_packing["Box No."] = df_packing["Box No."].ffill()

df_packing[["Product Number", "Qty"]] = df_packing["Packing List Lines"].apply(
    lambda line: pd.Series(extract_item_qty(line))
)
df_packing.drop(columns=['Packing List Lines'], inplace=True)

# === Output ===
print(f'Total Items:', len(df_packing))
print("\n📦 FULL PACKING LIST SECTION:")
print(df_packing)


Total Items: 24

📦 FULL PACKING LIST SECTION:
      Box No.                        Product Number  Qty
0    20- 1-10       N8108GC-QD-i7G965-A4-MII-CF4-50   20
1      20- 11                     POC-330-AEI01-100    1
2      20- 11                  Cbl-DB9F-3DB9M-15CM2    1
3      20- 12                   NRU-162S-AWP-RH(EA)    1
4      20- 12                   NRU-230V-AWP-JAO64G    1
5      20- 12        M.280-SSD-2TB-PCIe44-TLC5ET-TD   10
6      20- 12                     SSD-1TB-TLC5ET-TD   10
7      20- 12               Ant-RP_SMAM-WiFi-196MM1   50
8      20- 12    AccsyBx-Cardholder-9160GC-2000EAda    2
9      20- 12                        Dmpbr-Nuvo9160    2
10     20- 12                          M.2-LTE-7455   30
11     20- 12                      Wmkit-H-Nuvo9501    2
12     20- 13                             mPCIe-M2E   10
13     20- 13                          PCIe-10G550X   10
14     20- 13  AccsyBx-PanelKits-Nuvo-9000LP-AUT-KS    8
15     20- 13      AccsyBx-PanelKits-POC-4

## Generate Receiving Log (Expend by Qty)

In [ ]:
import pandas as pd

# List of prefixes to skip expansion
skip_prefixes = ("Cbl", "DIN", "AccsyBx", "Gpubr", "Ant", "FK", "TB")

# Add flag for whether to expand
df_packing["Expand"] = ~df_packing["Product Number"].str.startswith(skip_prefixes)

# Store original row order
df_packing["OriginalIndex"] = df_packing.index

# Build expanded list
expanded_rows = []

for _, row in df_packing.iterrows():
    if row["Expand"]:
        for _ in range(int(row["Qty"])):
            expanded_rows.append({
                "Box No.": row["Box No."],
                "Product Number": row["Product Number"],
                "Qty": pd.NA,  # use NA for blank (nullable int column)
                "OriginalIndex": row["OriginalIndex"]
            })
    else:
        expanded_rows.append({
            "Box No.": row["Box No."],
            "Product Number": row["Product Number"],
            "Qty": int(row["Qty"]),  # keep as integer
            "OriginalIndex": row["OriginalIndex"]
        })

# Create final DataFrame
final_df = pd.DataFrame(expanded_rows)

# Ensure Qty column is treated as nullable integer
final_df["Qty"] = final_df["Qty"].astype("Int64")

# Sort back to original order
final_df.sort_values("OriginalIndex", inplace=True)
final_df.drop(columns=["OriginalIndex", "Expand"], errors='ignore', inplace=True)
final_df.reset_index(drop=True, inplace=True)

# Reformat the columns to match your template
final_df["Box #"] = final_df["Box No."]
final_df["Part#"] = final_df["Product Number"]
final_df["POD#"] = ""   # Leave blank
final_df["SN#"] = ""    # Leave blank

# Reorder and select only the needed columns
output_df = final_df[["Box #", "POD#", "Part#", "SN#", "Qty"]]

# Save to Excel
output_path = r"C:\Users\Admin\OneDrive - neousys-tech\Desktop\Output.xlsx"
output_df.to_excel(output_path, index=False)

# === Output ===
print(f'Total Items:', len(output_df))
print("\n📦 FULL PACKING LIST SECTION:")
print(output_df)

Total Items: 458

📦 FULL PACKING LIST SECTION:
         Box # POD#                                Part# SN#   Qty
0      77- 1-6                  SEMIL-2007-i9IC14-65W-DS      <NA>
1      77- 1-6                  SEMIL-2007-i9IC14-65W-DS      <NA>
2      77- 1-6                  SEMIL-2007-i9IC14-65W-DS      <NA>
3      77- 1-6                  SEMIL-2007-i9IC14-65W-DS      <NA>
4      77- 1-6                  SEMIL-2007-i9IC14-65W-DS      <NA>
5      77- 1-6                  SEMIL-2007-i9IC14-65W-DS      <NA>
6     77- 7-13                       Nuvo-9002E-IFCN-CF2      <NA>
7     77- 7-13                       Nuvo-9002E-IFCN-CF2      <NA>
8     77- 7-13                       Nuvo-9002E-IFCN-CF2      <NA>
9     77- 7-13                       Nuvo-9002E-IFCN-CF2      <NA>
10    77- 7-13                       Nuvo-9002E-IFCN-CF2      <NA>
11    77- 7-13                       Nuvo-9002E-IFCN-CF2      <NA>
12    77- 7-13                       Nuvo-9002E-IFCN-CF2      <NA>
13    77- 7-13 

## Upload receiving log to DB

In [18]:
import logging
from flask import Flask, jsonify # type: ignore
from flask_sqlalchemy import SQLAlchemy # type: ignore
import pandas as pd # type: ignore
from sqlalchemy import Column, String, Float, Date, Integer



def load_receiving_log(path_to_xlsm: str, engine, dry_run=False):
    import pandas as pd

    # Load only the 'Receiving' sheet
    df = pd.read_excel(path_to_xlsm, sheet_name="Receiving")

    # Remove rows where all critical fields are blank (before renaming)
    df = df.dropna(subset=['Date', 'Inv# ', 'Box #', 'POD#', 'Part#', 'SN#'], how='all')

    # Rename columns to standardized names
    df.rename(columns={
        'Date': 'entry_date',
        'Inv# ': 'invoice_number',
        'Box #': 'box_number',
        'POD#': 'pod_number',
        'Part#': 'part_number',
        'SN#': 'serial_number',
        'QTY': 'quantity'
    }, inplace=True)

    # Normalize types and strip spaces
    df['quantity'] = df['quantity'].fillna(1).astype(float)
    df['entry_date'] = pd.to_datetime(df['entry_date'], errors='coerce')
    df['serial_number'] = df['serial_number'].astype(str).str.strip().replace("nan", "NA")

    string_cols = ['invoice_number', 'box_number', 'pod_number', 'part_number', 'serial_number']
    for col in string_cols:
        df[col] = df[col].astype(str).str.strip()

    # Print preview after cleanup
    print(f"🧾 Cleaned DataFrame Preview:\n{df.tail(25)}")

    # Load existing data for deduplication
    existing = pd.read_sql("""
        SELECT entry_date, invoice_number, box_number, pod_number, part_number, serial_number, quantity
        FROM receiving_log
    """, engine)

    existing['entry_date'] = pd.to_datetime(existing['entry_date'], errors='coerce')
    for col in string_cols:
        existing[col] = existing[col].astype(str).str.strip()
    existing['quantity'] = existing['quantity'].astype(float)


    # 🔍 Deduplicate based on key fields
    key_cols = ['entry_date', 'invoice_number', 'box_number', 'pod_number', 'part_number', 'serial_number', 'quantity']
    merged = df.merge(existing, how='left', indicator=True, on=key_cols) ## Only if all the seven columns are matched, otherwise we take it as new line
    print(f"🧾 merged:\n{merged.tail(25)}")
    new_rows = merged[merged['_merge'] == 'left_only'].drop(columns=['_merge'])

    print(f"🟡 Dry Run: {len(new_rows)} new rows would be inserted (out of {len(df)} total).")

    if not dry_run:
        new_rows.to_sql('receiving_log', engine, if_exists='append', index=False, method='multi')
        print("✅ Data inserted.")
    else:
        print("🚫 Dry run mode — no data inserted.")
        print("🔍 Preview of rows to be inserted:")
        print(new_rows.head(10))


# Configure Supabase
logging.basicConfig(level=logging.INFO)

app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = (
    "postgresql://postgres.avcznjglmqhmzqtsrlfg:"
    "Czheyuan0227@aws-0-us-east-2.pooler.supabase.com:6543/postgres"
)
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False


db = SQLAlchemy()

# Hook your existing db object up to this app
db.init_app(app)

class ReceivingLog(db.Model):
    __tablename__   = 'receiving_log'
    id = Column(Integer, primary_key=True, autoincrement=True)
    serial_number   = db.Column(db.String(255), primary_key=True)
    entry_date      = db.Column(db.Date)
    invoice_number  = db.Column(db.String(255))
    box_number      = db.Column(db.String(255))
    pod_number      = db.Column(db.String(255))
    part_number     = db.Column(db.String(255))
    quantity        = db.Column(db.Float)
    reference     = db.Column('Reference', db.Text)  


if __name__ == '__main__':
    # 1️⃣ Push the Flask context so db.engine is wired up:
    with app.app_context():
        # 2️⃣ Ensure the receiving_log table exists
        db.create_all()

        # 3️⃣ Now call your loader, handing it db.engine
        load_receiving_log(
            r"C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\01 Incoming\Receiving Log_ZC_2.0.xlsm",
            db.engine
        )

🧾 Cleaned DataFrame Preview:
     entry_date invoice_number  box_number  pod_number  \
3206        NaT    IN250917003  20--15--16  POD-251057   
3207        NaT    IN250917003  20--15--16  POD-251057   
3208        NaT    IN250917003  20--15--16  POD-251057   
3209        NaT    IN250917003  20--15--16  POD-251057   
3210        NaT    IN250917003  20--15--16  POD-251057   
3211        NaT    IN250917003  20--15--16  POD-251057   
3212        NaT    IN250917003  20--15--16  POD-251057   
3213        NaT    IN250917003  20--15--16  POD-251057   
3214        NaT    IN250917003  20--15--16  POD-251057   
3215        NaT    IN250917003  20--15--16  POD-251057   
3216        NaT    IN250917003  20--15--16  POD-251057   
3217        NaT    IN250917003  20--15--16  POD-251057   
3218        NaT    IN250917003  20--15--16  POD-251057   
3219        NaT    IN250917003  20--15--16  POD-251057   
3220        NaT    IN250917003  20--15--16  POD-251057   
3221        NaT    IN250917003  20--15--16 

## Generate Incoming Form

In [20]:
import logging
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import Column, String, Float, Date, Integer


# Configure Flask app and SQLAlchemy
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = (
    "postgresql://postgres.avcznjglmqhmzqtsrlfg:"
    "Czheyuan0227@aws-0-us-east-2.pooler.supabase.com:6543/postgres"
)
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

# Define the model
class ReceivingLog(db.Model):
    __tablename__ = 'receiving_log'
    id = Column(Integer, primary_key=True, autoincrement=True)
    serial_number = Column(String(255))
    entry_date    = db.Column(db.Date)
    invoice_number= db.Column(db.String(255))
    box_number    = db.Column(db.String(255))
    pod_number    = db.Column(db.String(255))
    part_number   = db.Column(db.String(255))
    quantity      = db.Column(db.INT)
    reference      = db.Column('Reference', db.Text) 

# Query the database inside the app context
with app.app_context():
    rows = ReceivingLog.query.filter_by(invoice_number='IN250917003').all()

    df = pd.DataFrame([{
        "Part Number": r.part_number,
        "Quantity": r.quantity,
        "Serial Number": r.serial_number,
        "Box Number": r.box_number,
        "POD Number": r.pod_number,
        
    } for r in rows])

    df['Serial Number'] = df['Serial Number'].fillna("NA").astype(str).str.strip()
    
# Step 1: Group and aggregate Quantity, preserving order
df_reset = df.reset_index()
grp = (
    df_reset
    .groupby(["Box Number", "Part Number", "POD Number"], as_index=False)
    .agg(
        Quantity=("Quantity", "sum"),
        first_idx=("index", "min")
    )
    .sort_values("first_idx")
    .drop(columns="first_idx")
    .reset_index(drop=True)
)

# Step 2: Build a serial number map per group (ignore exact duplicates, but keep 'NA')
serial_map = (
    df.groupby(["Box Number", "Part Number", "POD Number"])["Serial Number"]
    .apply(lambda x: ", ".join(x.astype(str).dropna().unique()))
    .reset_index()
    .rename(columns={"Serial Number": "Serial Numbers"})
)

# Step 3: Merge serial numbers into the main grouped result
grp_with_serials = grp.merge(
    serial_map,
    on=["Box Number", "Part Number", "POD Number"],
    how="left"
)

# Step 4: Reorder columns (optional)
grp_with_serials = grp_with_serials[
    ["Part Number", "Quantity", "Serial Numbers", "Box Number", "POD Number"]
]

print(f'Total Lines:', len(grp_with_serials))

# Step 5: Display in Jupyter notebook
from IPython.display import display, HTML
display(HTML(grp_with_serials.to_html(index=False)))



Total Lines: 24


Part Number,Quantity,Serial Numbers,Box Number,POD Number
N8108GC-QD-i7G965-A4-MII-CF4-50,20,"P3700497, P3700498, P3700499, P3700500, P3700501, P3700502, P3700503, P3700504, P3700505, P3700506, P3700507, P3700508, P3700509, P3700510, P3700511, P3700512, P3700513, P3700514, P3700515, P3700516",20--1--10,POD-250847
POC-330-AEI01-100,1,P3700611,20--11,POD-251057
Cbl-DB9F-3DB9M-15CM2,1,NA,20--11,POD-251057
NRU-162S-AWP-RH(EA),1,P3700133,20--12,POD-251094
NRU-230V-AWP-JAO64G,1,P3700524,20--12,POD-251215
M.280-SSD-2TB-PCIe44-TLC5ET-TD,10,"J563280004, J563280020, J563280062, J563280053, J563280047, J563280058, J563280061, J563280010, J563280021, J563280002",20--12,POD-251231
SSD-1TB-TLC5ET-TD,10,"J495100006, J495100014, J495100027, J495100026, J495100005, J495100009, J495100030, J495100004, J495100028, J495100029",20--12,POD-251248
Ant-RP_SMAM-WiFi-196MM1,50,NA,20--12,POD-251248
AccsyBx-Cardholder-9160GC-2000EAda,2,NA,20--12,POD-251250
Dmpbr-Nuvo9160,2,NA,20--12,POD-251250


In [22]:
from docx import Document

def write_table_to_word(df, doc_path, output_path):
    # Load the existing Word document
    doc = Document(doc_path)

    # Add a new table with headers + rows
    table = doc.add_table(rows=1, cols=len(df.columns))
    # table.style = 'Table Grid'

    # Write the header row
    hdr_cells = table.rows[0].cells
    for i, column_name in enumerate(df.columns):
        hdr_cells[i].text = column_name

    # Write the data rows
    for _, row in df.iterrows():
        row_cells = table.add_row().cells
        for i, val in enumerate(row):
            row_cells[i].text = str(val)

    # Save to new Word file
    doc.save(output_path)
    print("✅ New table written to:", output_path)

# Example usage:
doc_path = r"C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\01 Incoming\Incoming Form Template.docx"
output_path = r"C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\01 Incoming\I07-070125-TaiwanHQ-IN240628006(Template).docx"
write_table_to_word(grp_with_serials, doc_path, output_path)



✅ New table written to: C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\01 Incoming\I07-070125-TaiwanHQ-IN240628006(Template).docx
